<a href="https://colab.research.google.com/github/IshaSharma222/Bio-NER/blob/master/Word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
(train_data, test_data), info = tfds.load(MODEL_PATH, split = (tfds.Split.TRAIN, tfds.Split.TEST), with_info=True, as_supervised=True)  #load train and test dataset

In [ ]:
encoder=info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [ ]:
train_batches=train_data.shuffle(1000).padded_batch(10)
test_batches=test_data.shuffle(1000).padded_batch(10)

In [ ]:
train_batch, train_labels=next(iter(train_batches))

In [ ]:
train_batch.numpy()

array([[7969, 5321, 7114, ...,   38, 2293, 2760],
       [  19,  873, 3473, ...,    0,    0,    0],
       [ 133,  279,   86, ...,    0,    0,    0],
       ...,
       [1208, 1758, 4108, ...,    0,    0,    0],
       [5440,  131, 5617, ...,    0,    0,    0],
       [7459, 7969, 7455, ...,    0,    0,    0]])

In [ ]:
# CREATING A Continuous bag of words type model

In [ ]:
embedding_dim=16
model= keras.Sequential([layers.Embedding(encoder.vocab_size, embedding_dim), layers.GlobalAveragePooling1D(),layers.Dense(16, activation='relu'), layers.Dense(1)])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history=model.fit(train_batches,epochs=10, validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.5404 - accuracy: 0.6638 - val_loss: 0.4279 - val_accuracy: 0.8150
Epoch 2/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.3002 - accuracy: 0.8722 - val_loss: 0.3899 - val_accuracy: 0.7850
Epoch 3/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.2405 - accuracy: 0.9039 - val_loss: 0.4062 - val_accuracy: 0.8350
Epoch 4/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2065 - accuracy: 0.9199 - val_loss: 0.3266 - val_accuracy: 0.8700
Epoch 5/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.1832 - accuracy: 0.9302 - val_loss: 0.3813 - val_accuracy: 0.8800
Epoch 6/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.1669 - accuracy: 0.9396 - val_loss: 0.3805 - val_accuracy: 0.8500
Epoch 7/10
2500/2500 [==============================] - 14s 6ms/step - loss: 0.1513 - accuracy: 0.9460 - val_loss: 0.4440 - val_ac

In [ ]:
# RETRIEVE THE LEARNED EMBEDDINGS

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(8185, 16)


In [ ]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# Writing to our local disk

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>